#Analyzing physical activity monitor data

TODO intro

## Data

TODO

To convert from sas transport file format with extension`.xpt` to CSV we can use `xport` module from python PyPI packages:
```bash
pip install xport
```
and use the `xport` module as a command-line tool to convert an XPT file to CSV file:

```bash
python -m xport paxraw_d.xpt > paxraw_d.csv
```

## Reading the data

In [ ]:
val spark = sparkSession

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@40f2ef36


In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._


###Physical activity monitor data

In [ ]:
val PaxSchema = StructType(
    StructField("SEQN", FloatType, false) ::
    StructField("PAXSTAT", FloatType, false) ::
    StructField("PAXCAL", FloatType, false) ::
    StructField("PAXDAYSAS", FloatType, false) ::
    StructField("PAXN", FloatType, false) ::
    StructField("PAXHOUR", FloatType, false) ::
    StructField("PAXMINUT", FloatType, false) ::
    StructField("PAXINTEN", FloatType, false) ::
    StructField("PAXSTEP", StringType, true) :: Nil
)

TODO note on `nan` values in `PAXSTEP` column

In [ ]:
val PaxDF = spark.read
  .format("csv")
  .schema(PaxSchema)
  .option("header", true)
  .load("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/paxraw_d.csv")
  .withColumn("PAXSTEP", when($"PAXSTEP" === "nan", null).otherwise($"PAXSTEP".cast(DoubleType)))
  .select($"SEQN".cast(IntegerType),
          $"PAXSTAT".cast(IntegerType),
          $"PAXCAL".cast(IntegerType),
          $"PAXDAYSAS".cast(IntegerType),
          $"PAXN".cast(IntegerType),
          $"PAXHOUR".cast(IntegerType),
          $"PAXMINUT".cast(IntegerType),
          $"PAXINTEN".cast(IntegerType),
          $"PAXSTEP".cast(IntegerType))
  .filter(!isnull($"PAXSTEP"))

In [ ]:
PaxDF.write.format("parquet").save("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/paxraw_d.parquet")

In [ ]:
val PaxDF = spark.read
  .format("parquet")
  .load("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/paxraw_d.parquet")
  .withColumn("datetime", concat($"PAXDAYSAS", lit(".01.2005 "), $"PAXHOUR", lit(":"), $"PAXMINUT"))
  .withColumn("time", unix_timestamp($"datetime", "d.MM.yyyy HH:mm"))
  .withColumn("datetime", from_unixtime($"time"))

PaxDF: org.apache.spark.sql.DataFrame = [SEQN: int, PAXSTAT: int ... 9 more fields]


###Demographics Data

In [ ]:
val DemoDF = spark.read
  .format("csv")
  .option("header", true)
  .load("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/DEMO_D.csv")
  .select($"SEQN".cast(IntegerType), $"RIDAGEYR".cast(IntegerType))

DemoDF: org.apache.spark.sql.DataFrame = [SEQN: int, RIDAGEYR: int]


In [ ]:
DemoDF.limit(3).show

+-----+--------+
| SEQN|RIDAGEYR|
+-----+--------+
|31127|       0|
|31128|      11|
|31129|      15|
+-----+--------+



Let's take a closer look at the data.

In [ ]:
PaxDF.describe("PAXSTEP").show

+-------+------------------+
|summary|           PAXSTEP|
+-------+------------------+
|  count|          74874033|
|   mean|22.429516051312476|
| stddev| 708.6893633662772|
|    min|                 0|
|    max|             32767|
+-------+------------------+



`PAXSTEP` - is the step count per minute recorded by the physical activity monitor.
The values like over `500` steps per minutes seems unreliable to me.
Let's excluse those devices that have recorded more than `500` steps per minute.

In [ ]:
PaxDF.where($"PAXSTEP" > 500).select($"SEQN").distinct.count

res5: Long = 166


166

In [ ]:
val broadcastedBlackList = spark.sparkContext.broadcast(
  PaxDF.where($"PAXSTEP" > 500).select($"SEQN").distinct
  .collect.map(_(0).asInstanceOf[Int]).toSet
)

broadcastedBlackList: org.apache.spark.broadcast.Broadcast[scala.collection.immutable.Set[Int]] = Broadcast(245)


In [ ]:
def inBlacklistUDF = udf((seqNum: Int) => {
  broadcastedBlackList.value.contains(seqNum)
})

inBlacklistUDF: org.apache.spark.sql.expressions.UserDefinedFunction


In [ ]:
val PaxUnreliable = PaxDF.where(inBlacklistUDF($"SEQN"))

PaxUnreliable: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [SEQN: int, PAXSTAT: int ... 9 more fields]


In [ ]:
val PaxReliable = PaxDF.where(!inBlacklistUDF($"SEQN"))

PaxReliable: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [SEQN: int, PAXSTAT: int ... 9 more fields]


In [ ]:
PaxReliable.count

res12: Long = 73433881


73433881

In [ ]:
PaxReliable.select($"SEQN").distinct.count

res14: Long = 7289


7289

val seqNums = PaxReliable.select($"SEQN").distinct
  .sample(false, 0.01)
  .limit(10)
  .collect
  .map(_(0).asInstanceOf[Int]).toList

TODO show for both Reliable and Unreliable data.

In [ ]:
CustomPlotlyChart(PaxReliable.where($"SEQN" === seqNums(1)),
                  layout="""{title: 'Physical activity monitor data', 
                           yaxis: {title: 'Device Step Count per 1 min'},
                           showlegend: false}""",
                  dataOptions="""{
                    colorscale: 'Electric',
                    autocolorscale: true
                  }""",
                  dataSources="""{
                    x: 'datetime',
                    y: 'PAXSTEP'
                  }""",
                 maxPoints=2000)

res49: notebook.front.widgets.charts.CustomPlotlyChart[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = <CustomPlotlyChart widget>


Let's see on steps count distribution

TODO show for both Reliable and Unreliable data.

In [ ]:
CustomPlotlyChart(PaxReliable.sample(withReplacement=false, 0.001),
                  layout="""{title: 'Steps count distribution', 
                             yaxis: {type: 'log'},
                             xaxis: {title: 'Step Count per minute'},
                             bargap: 0.02}""",
                  dataOptions="{type: 'histogram', opacity: 0.7}",
                  dataSources="{x: 'PAXSTEP'}",
                  maxPoints=5000)

res9: notebook.front.widgets.charts.CustomPlotlyChart[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = <CustomPlotlyChart widget>


## Bucketize

*REPEAT for intevals between array([ 31128,  31672,  32217.05263158,  32761.57894737,
        33306.10526316,  33850.63157895,  34395.15789474,  34939.68421053,
        35484.21052632,  36028.73684211,  36573.26315789,  37117.78947368,
        37662.31578947,  38206.84210526,  38751.36842105,  39295.89473684,
        39840.42105263,  40384.94736842,  40929.47368421,  41474.        ])*

In [ ]:
import org.apache.spark.ml.feature.Bucketizer

val splits = Array(0, 1, 5, 12, 20, 30, 40, 60, 90, 120, Double.PositiveInfinity)

val bucketizer = new Bucketizer()
  .setInputCol("PAXSTEP")
  .setOutputCol("activityLevel")
  .setSplits(splits)

import org.apache.spark.ml.feature.Bucketizer
splits: Array[Double] = Array(0.0, 1.0, 5.0, 12.0, 20.0, 30.0, 40.0, 60.0, 90.0, 120.0, Infinity)
bucketizer: org.apache.spark.ml.feature.Bucketizer = bucketizer_9151115cafb2


In [ ]:
val bucketedPax = bucketizer
  .transform(PaxReliable
             .withColumn("totalSteps", $"PAXSTEP".cast(LongType))
             .withColumn("PAXSTEP", $"PAXSTEP".cast(DoubleType)))
  .withColumn("activityLevel", $"activityLevel".cast(IntegerType))

bucketedPax: org.apache.spark.sql.DataFrame = [SEQN: int, PAXSTAT: int ... 11 more fields]


In [ ]:
bucketedPax.select($"activityLevel").distinct.orderBy($"activityLevel").show

+-------------+
|activityLevel|
+-------------+
|            0|
|            1|
|            2|
|            3|
|            4|
|            5|
|            6|
|            7|
|            8|
|            9|
+-------------+



## Building a Transition Matrix

In [ ]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window


1. use `Window Function` to collect previous minute activity

In [ ]:
val windowSpec = Window.partitionBy("SEQN").orderBy("time")

windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@12ca436c


In [ ]:
val df = bucketedPax
  .select($"SEQN", $"totalSteps", $"activityLevel", $"time")
  .withColumn("previousMinuteActivity", lag("activityLevel", 1).over(windowSpec))
  .withColumn("previousMinuteActivity", when(isnull($"previousMinuteActivity"), -1).otherwise($"previousMinuteActivity"))

df: org.apache.spark.sql.DataFrame = [SEQN: int, totalSteps: bigint ... 3 more fields]


In [ ]:
def initTransitionMatrix = udf{ (currentActivityLevel: Int, previousActivityLevel: Int, size: Int) => {
  val W = Array.fill(size, size)(0.0)
  if (previousActivityLevel >= 0)
    W.updated(currentActivityLevel, W(currentActivityLevel).updated(previousActivityLevel, 1.0))
  else
    W
}}

initTransitionMatrix: org.apache.spark.sql.expressions.UserDefinedFunction


In [ ]:
val dfW = df.withColumn("W", initTransitionMatrix($"activityLevel", $"previousMinuteActivity", lit(10)))

dfW: org.apache.spark.sql.DataFrame = [SEQN: int, totalSteps: bigint ... 4 more fields]


In [ ]:
case class RespondentTrMatrix(seqn: Int, totalSteps: Long, totalCount: Long, W: Array[Array[Double]])

defined class RespondentTrMatrix


In [ ]:
val trMatrixDS = dfW.select($"SEQN", $"totalSteps", lit(1L).as("totalCount"), $"W").as[RespondentTrMatrix]

trMatrixDS: org.apache.spark.sql.Dataset[RespondentTrMatrix] = [SEQN: int, totalSteps: bigint ... 2 more fields]


In [ ]:
trMatrixDS.write.format("parquet").save("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/10_tr_matrix_init.parquet")

In [ ]:
val initTrMatrixDS = spark.read
  .format("parquet")
  .load("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/10_tr_matrix_init.parquet")
  .as[RespondentTrMatrix]


initTrMatrixDS: org.apache.spark.sql.Dataset[RespondentTrMatrix] = [SEQN: int, totalSteps: bigint ... 2 more fields]


In [ ]:

val sumTrMatrixDS = initTrMatrixDS.rdd
  .map(l => (l.seqn, l))
  .reduceByKey((l, r) => {
    val elementWiseArraySum = (a: Array[Double], b: Array[Double]) => {
      a.zip(b).map { case (x, y) => x + y }
    }
    val elementWiseMatrixSum = (c: Array[Array[Double]], d: Array[Array[Double]]) => {
      c.zip(d).map { case (x, y) => elementWiseArraySum(x, y) }
    }
    RespondentTrMatrix(l.seqn, l.totalSteps + r.totalSteps, l.totalCount + r.totalCount, elementWiseMatrixSum(l.W, r.W)) 
  })
  .map(r => {
    val trMatrix = r._2
    trMatrix.copy(W = trMatrix.W.map(_.map(_ / trMatrix.totalCount)))
  })
  .toDS

sumTrMatrixDS: org.apache.spark.sql.Dataset[RespondentTrMatrix] = [seqn: int, totalSteps: bigint ... 2 more fields]


In [ ]:
sumTrMatrixDS
  .write
  .format("parquet")
  .save("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/10_tr_matrix_d.parquet")

In [ ]:
val computedTrMatrixDS = spark.read
  .format("parquet")
  .load("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/10_tr_matrix_d.parquet")
  .as[RespondentTrMatrix]


computedTrMatrixDS: org.apache.spark.sql.Dataset[RespondentTrMatrix] = [seqn: int, totalSteps: bigint ... 2 more fields]


In [ ]:
CustomPlotlyChart(computedTrMatrixDS.where($"totalSteps" > 1500).toDF,
                  layout="""{title: 'Total steps count distribution', 
                             xaxis: {title: 'Step Count per week'},
                             bargap: 0.02}""",
                  dataOptions="{type: 'histogram', opacity: 0.7}",
                  dataSources="{x: 'totalSteps'}",
                  maxPoints=8000)

res47: notebook.front.widgets.charts.CustomPlotlyChart[org.apache.spark.sql.DataFrame] = <CustomPlotlyChart widget>


In [ ]:
computedTrMatrixDS.describe("totalSteps")

res29: org.apache.spark.sql.DataFrame = [summary: string, totalSteps: string]


In [ ]:
val trMatrix = computedTrMatrixDS.sample(false, 0.1, 221).limit(1).collect.head.W

trMatrix: Array[Array[Double]] = Array(Array(0.5608134920634921, 0.03283730158730159, 0.00753968253968254, 0.0036706349206349206, 0.002281746031746032, 0.0016865079365079366, 3.968253968253968E-4, 1.984126984126984E-4, 0.0, 0.0), Array(0.03194444444444444, 0.06170634920634921, 0.027182539682539683, 0.009722222222222222, 0.004563492063492064, 0.0025793650793650793, 0.0025793650793650793, 5.952380952380953E-4, 0.0, 0.0), Array(0.009523809523809525, 0.026686507936507935, 0.030456349206349206, 0.014682539682539682, 0.007738095238095238, 0.0035714285714285713, 0.0030753968253968253, 6.944444444444445E-4, 3.968253968253968E-4, 0.0), Array(0.0036706349206349206, 0.00873015873015873, 0.014682539682539682, 0.014186507936507936, 0.007142857142857143, 0.00496031746031746, 0.00376984126984127, 4.96...

In [ ]:
val trMatrixPlotData = trMatrix
                       .zipWithIndex.toSeq.toDF("transitions", "toActivityLevel").withColumn("fromActivityLevel", $"toActivityLevel")

trMatrixPlotData: org.apache.spark.sql.DataFrame = [transitions: array<double>, toActivityLevel: int ... 1 more field]


In [ ]:
CustomPlotlyChart(trMatrixPlotData,
                  layout="""{title: 'Physical activity Transition matrix',
                             xaxis: {title: 'from physical activity level'}, 
                             yaxis: {title: 'to physical activity level'},
                             width: 600, height: 600}""",
                  dataOptions="""{type: 'heatmap', 
                                  colorscale: 'Viridis',
                                  reversescale: false,
                                  colorbar: {
                                    title: 'Probability',
                                    tickmode: 'array',
                                    tickvals: [0, 0.02, 0.04, 0.06, 0.08, 0.1],
                                    ticktext: ['0', '0.02', '0.04', '0.06', '0.08', '>0.1']
                                  },
                                  zmin: 0.0, zmax: 0.10}""",
                  dataSources="{x: 'fromActivityLevel', y: 'toActivityLevel', z: 'transitions'}")

res7: notebook.front.widgets.charts.CustomPlotlyChart[org.apache.spark.sql.DataFrame] = <CustomPlotlyChart widget>


entries total

## PCA (unsupervised learning)

In [ ]:
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.feature.PCA

import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.feature.PCA


In [ ]:
val flattenTrMatrixDF = computedTrMatrixDS.where($"totalSteps" > 1500).rdd
  .map(l => (l.seqn, Vectors.dense(l.W.flatten)))
  .toDF("SEQN", "features")
  .join(DemoDF, "SEQN")
  .where($"RIDAGEYR" >= 35)

flattenTrMatrixDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [SEQN: int, features: vector ... 1 more field]


In [ ]:
val pca = new PCA()
  .setInputCol("features")
  .setOutputCol("pcaFeatures")
  .setK(3)
  .fit(flattenTrMatrixDF)

val withLocomotorPCA = pca.transform(flattenTrMatrixDF).select("SEQN", "pcaFeatures", "RIDAGEYR")

pca: org.apache.spark.ml.feature.PCAModel = pca_5bb58ccfdb42
withLocomotorPCA: org.apache.spark.sql.DataFrame = [SEQN: int, pcaFeatures: vector ... 1 more field]


In [ ]:
withLocomotorPCA.limit(3).show(false)

+-----+--------------------------------------------------------------+--------+
|SEQN |pcaFeatures                                                   |RIDAGEYR|
+-----+--------------------------------------------------------------+--------+
|41234|[-0.6066457102517638,-0.1778212116439929,0.06494364878932443] |78      |
|31506|[-0.6370533924192854,-0.11131860492400716,0.04740517634107428]|73      |
|34578|[-0.5381452731736295,-0.09158440756746544,0.08147225657803708]|55      |
+-----+--------------------------------------------------------------+--------+



In [ ]:
def getItemUDF = udf{ (vec: Vector, idx: Int) => vec(idx)}

getItemUDF: org.apache.spark.sql.expressions.UserDefinedFunction


In [ ]:
val locomotorPCvsAge = withLocomotorPCA
  .select($"SEQN", $"RIDAGEYR".as("age"),
          getItemUDF($"pcaFeatures", lit(0)).as("PC1"),
          getItemUDF($"pcaFeatures", lit(1)).as("PC2"),
          getItemUDF($"pcaFeatures", lit(2)).as("PC3"))

locomotorPCvsAge: org.apache.spark.sql.DataFrame = [SEQN: int, age: int ... 3 more fields]


In [ ]:
val matrixPCvsAge = locomotorPCvsAge.groupBy($"age").agg(
  mean($"PC1").as("meanPC1"), stddev($"PC1").as("stdPC1"),
  mean($"PC2").as("meanPC2"), stddev($"PC2").as("stdPC2"),
  mean($"PC3").as("meanPC3"), stddev($"PC3").as("stdPC3")
)

matrixPCvsAge: org.apache.spark.sql.DataFrame = [age: int, meanPC1: double ... 5 more fields]


In [ ]:
CustomPlotlyChart(matrixPCvsAge.where($"age" >= 15).orderBy($"age"),
                  layout="""{title: 'PC projections vs Age', 
                           xaxis: {title: 'Chronological age'},
                           yaxis: {title: 'PC projection'},
                           showlegend: false}""",
                  dataOptions="""{
                    type: 'scatter',
                    line: {width: 2},
                    error_y: {type: 'data', visible: true, thickness: 0.5, width: 0}
                  }""",
                  dataSources="""{
                    x: 'age',
                    y: 'meanPC2',
                    error_y: {array: 'stdPC2'}
                  }""")

res63: notebook.front.widgets.charts.CustomPlotlyChart[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = <CustomPlotlyChart widget>


entries total

In [ ]:
CustomPlotlyChart(
  locomotorPCvsAge.where($"age" >= 35), 
  layout="""{
        title: 'PCA of the Locomotor Transition Matrix',
        height: 900,
        xaxis: {title: 'PC1'},
        yaxis: {title: 'PC2'},
        hovermode: 'closest'
    }""",
  dataOptions="""{
    mode: 'markers',
    type: 'scatter',
    marker: {
        sizemode: 'area',
        size: 12,
        opacity: 0.75,
        colorscale: 'Jet',
        reversescale: true,
        colorbar: {
          title: 'Age',
          thickness: 8.0
        }
    }}""",
  dataSources="""{
    x: 'PC1',
    y: 'PC2',
    marker: {color: 'age'}}""",
  maxPoints=3000)

res75: notebook.front.widgets.charts.CustomPlotlyChart[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = <CustomPlotlyChart widget>
